In [1]:
import numpy as np
import pandas as pd
import pickle
import os

In [2]:
# Add the python path to the folder containing some custom packages.
import sys
sys.path.insert(0, "../../packages/")
from LagsCreator.LagsCreator import LagsCreator
from NestedCV.NestedCV import NestedCV

In [3]:
# Create the workspace folder for storing training and test points.
dir_data = "./data_xgboost"
os.makedirs(dir_data)

In [4]:
# Select the countries to consider for the creation of training and test points.
COUNTRIES = ["Yemen"]
# Define the name of indicator we want to predict.
# We have to rename the indicator 'FCG <= 2' to 'FCG'.
TARGET = "FCG"
# Define the number of days we want to learn to predict for the target variable.
TEST_SIZE = 30
# Define the number of total split we want to evaluate using our nested cross validation method.
NUMBER_OF_SPLITS = 7
# Define the time features we want to create for the input samples.
FEATURE_TIMES = ["Day", "Month", "Dayofweek", "Year"]
# Define the step between points during the creation of samples for training and test.
STEP_BETWEEN_POINTS = 1

In [5]:
# Save the parameters.
with open(dir_data + "/global_variables", "wb") as f:
    pickle.dump([TARGET, TEST_SIZE, FEATURE_TIMES, COUNTRIES, NUMBER_OF_SPLITS, STEP_BETWEEN_POINTS], f)

## Time-series dataset

In [6]:
# Load the time-series data of the Yemen country.
df_yemen = pd.read_csv("../../Dataset time-series/output_data/Yemen/Yemen.csv", header = [0, 1], index_col = 0)
df_yemen.index.name = "Datetime"
df_yemen.index = pd.to_datetime(df_yemen.index)
freq = "D"
df_yemen.index.freq = freq
df_yemen.columns = pd.MultiIndex.from_tuples(map(lambda x: ("Yemen", x[0], x[1]), df_yemen.columns), names = ["Country", "AdminStrata", "Indicator"])

In [7]:
df = df_yemen.copy()
# Consider the following dates.
df = df.loc["2018-01-01":"2020-08-31"]
# Select countries.
df = df[COUNTRIES]
# We have to rename the indicator 'FCG <= 2' to 'FCG' and 'rCSI >= 19' to 'rCSI' (XGBoost issue).
df.rename({"FCG <= 2": "FCG", "rCSI >= 19": "rCSI"}, axis = 1, level = 2, inplace = True)
df

Country                             Yemen                                      \
AdminStrata                         Abyan                                       
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%) Code   
Datetime                                                                        
2018-01-01                            NaN                            NaN   65   
2018-01-02                            NaN                            NaN   65   
2018-01-03                            NaN                            NaN   65   
2018-01-04                            NaN                            NaN   65   
2018-01-05                            NaN                            NaN   65   
...                                   ...                            ...  ...   
2020-08-27                            NaN                            NaN   65   
2020-08-28                            NaN                            NaN   65   
2020-08-29                            NaN                            NaN   65   
2020-08-30                            NaN                            NaN   65   
2020-08-31                     109.395086                     168.026904   65   

Country                                                               \
AdminStrata                                                            
Indicator   Exchange rate       FCG Fatalities        Lat        Lon   
Datetime                                                               
2018-01-01            NaN       NaN        NaN  13.704878  46.158142   
2018-01-02            NaN       NaN        NaN  13.704878  46.158142   
2018-01-03            NaN       NaN        NaN  13.704878  46.158142   
2018-01-04            NaN       NaN        NaN  13.704878  46.158142   
2018-01-05            NaN       NaN        NaN  13.704878  46.158142   
...                   ...       ...        ...        ...        ...   
2020-08-27            NaN  41.90447       48.0  13.704878  46.158142   
2020-08-28            NaN  40.88619       48.0  13.704878  46.158142   
2020-08-29            NaN  41.59444       48.0  13.704878  46.158142   
2020-08-30            NaN  41.90045       48.0  13.704878  46.158142   
2020-08-31       0.619324  42.76342       48.0  13.704878  46.158142   

Country                             ...                                   \
AdminStrata                         ...      Taizz                         
Indicator        NDVI NDVI Anomaly  ... Fatalities        Lat        Lon   
Datetime                            ...                                    
2018-01-01        NaN          NaN  ...        NaN  13.416517  43.778161   
2018-01-02        NaN          NaN  ...        NaN  13.416517  43.778161   
2018-01-03        NaN          NaN  ...        NaN  13.416517  43.778161   
2018-01-04        NaN          NaN  ...        NaN  13.416517  43.778161   
2018-01-05        NaN          NaN  ...        NaN  13.416517  43.778161   
...               ...          ...  ...        ...        ...        ...   
2020-08-27        NaN          NaN  ...       45.0  13.416517  43.778161   
2020-08-28        NaN          NaN  ...       35.0  13.416517  43.778161   
2020-08-29        NaN          NaN  ...       35.0  13.416517  43.778161   
2020-08-30        NaN          NaN  ...       34.0  13.416517  43.778161   
2020-08-31   0.174649   142.414507  ...       33.0  13.416517  43.778161   

Country                                                                 \
AdminStrata                                                              
Indicator        NDVI NDVI Anomaly Population Price cereals and tubers   
Datetime                                                                 
2018-01-01        NaN          NaN    3065034                      NaN   
2018-01-02        NaN          NaN    3065034                      NaN   
2018-01-03        NaN          NaN    3065034                      NaN   
2018-01-04        NaN          NaN    306503

## Creation training and test sets

In [8]:
# Define lags dictionary for each indicator.
lags_dict = dict()
# Define lags for each indicator.
lags_dict["3 Months Anomaly Rainfalls (%)"] = np.array([1,2,3]) 
lags_dict["1 Month Anomaly Rainfalls (%)"] = np.array([1,2,3]) 
lags_dict["Rainfalls (mm)"] = np.array([1,2,3]) 
lags_dict["Exchange rate"] = np.array([1,2,3]) 
lags_dict["Price cereals and tubers"] = np.array([1,2,3]) 
lags_dict["Fatalities"] = np.array([1,2,3,4]) 
lags_dict["NDVI Anomaly"] = np.array([1,2,3]) 
lags_dict["NDVI"] = np.array([1,2,3]) 
lags_dict["FCG"] = np.arange(1, 15)
lags_dict["rCSI"] = np.array([1,2,3,4]) 
lags_dict["Lat"] = np.array([1])
lags_dict["Lon"] = np.array([1])
lags_dict["Population"] = np.array([1])
lags_dict["Code"] = np.array([1])
lags_dict["Ramadan"] = np.array([1])

In [9]:
# Save the lags dictionary.
with open(dir_data + "/lags_dict", "wb") as fp:
    pickle.dump(lags_dict, fp)

In [10]:
# Create folder for containing training data.
os.makedirs(dir_data + "/train")
# Create folder for containing test data.
os.makedirs(dir_data + "/test")
for country in COUNTRIES:
    provinces = df[country].columns.get_level_values(0).unique()
    for province in provinces:
        os.makedirs(dir_data + "/train/%s/%s" % (country, province)) 
        os.makedirs(dir_data + "/test/%s/%s" % (country, province)) 

In [11]:
# Create the nested cross validation.
cv = NestedCV(NUMBER_OF_SPLITS, TEST_SIZE)
# Total nested cross validation.
SPLITS = cv.get_splits(df)
for split_number, (train, test) in SPLITS.items():
    print("Split %d: range of days to predict between %s - %s" % (split_number, str(test.index[0].date()), str(test.index[-1].date())))

Split 1: range of days to predict between 2020-02-01 - 2020-03-01
Split 2: range of days to predict between 2020-03-01 - 2020-03-30
Split 3: range of days to predict between 2020-04-01 - 2020-04-30
Split 4: range of days to predict between 2020-05-01 - 2020-05-30
Split 5: range of days to predict between 2020-06-01 - 2020-06-30
Split 6: range of days to predict between 2020-07-01 - 2020-07-30
Split 7: range of days to predict between 2020-08-01 - 2020-08-30


In [12]:
for split_number, (train, test) in SPLITS.items():
    print("Split %d. Please wait." % split_number)
    # Define the first multi-sites (countries).
    countries = train.columns.get_level_values(0).unique()
    for country in countries:
        train_country = train[country]
        # Define the second multi-sites (provinces).
        provinces = train_country.columns.get_level_values(0).unique()
        for province in provinces:
            creator = LagsCreator(train_country[province], lags_dictionary = lags_dict, target = TARGET, delay = True)
            for h in range(TEST_SIZE):
                # Training samples.
                X_train, y_train, X_test, features = creator.to_supervised(h = h+1, step = STEP_BETWEEN_POINTS, single_step = True, 
                                                                           return_dataframe = True, feature_time = FEATURE_TIMES, 
                                                                           dtype = float)

                # Train input and output.
                X_train.to_csv(dir_data + "/train/%s/%s/X_train_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 
                y_train.to_csv(dir_data + "/train/%s/%s/y_train_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 
                # Test input.
                X_test.to_csv(dir_data + "/test/%s/%s/X_test_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 

print("Complete!")

Split 1. Please wait.
Split 2. Please wait.
Split 3. Please wait.
Split 4. Please wait.
Split 5. Please wait.
Split 6. Please wait.
Split 7. Please wait.
Complete!
